In [1]:
import hopsworks
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier, plot_importance
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV

from xgboost import XGBClassifier, plot_importance
import xgboost as xgb
import joblib
import os
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from sklearn.metrics import confusion_matrix
import sklearn
import numpy as np


c:\Dev\Scalable ML\id2223\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

wine_fg = fs.get_feature_group(name="wine", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="wine",
                                  version=1,
                                  description="Read from wine dataset",
                                  labels=["quality"],
                                  query=query)




Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/190614
Connected. Call `.close()` to terminate connection gracefully.
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/190614/fs/190533/fv/wine/version/1


In [3]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.1)
X = pd.concat([X_train, X_test], axis = 0)
y = pd.concat([y_train, y_test], axis = 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

Finished: Reading data from Hopsworks, using Hive (1.97s) 


In [4]:
clf = RandomForestClassifier(n_estimators=2000, max_depth=12)

clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(confusion_matrix(y_test, predicted))
print(accuracy)

[[ 83  50   1]
 [ 44 113  21]
 [  2  46  34]]
0.583756345177665


In [5]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'wine_model' directory will be saved to the model registry. Create the dir, first.
model_dir="wine_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(clf, model_dir + "/wine_model.pkl")
#fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
version = 1
try:
    old_model = mr.get_model(name= "wine_model",
                 version=version)
    old_model.delete()
    print("deleted old version",version,"of model")
except:
    print("Unable to retrieve old model for replacement")
    
wine_model = mr.python.create_model(
    name="wine_model", 
    version=version,
    metrics={"accuracy" : accuracy},
    model_schema=model_schema,
    description="White Wine Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
wine_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.
deleted old version 1 of model


Model export complete: 100%|██████████| 6/6 [02:36<00:00, 26.13s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/190614/models/wine_model/1


Model(name: 'wine_model', version: 1)